# AME Project 1

In [15]:
# Import packeages 
import pandas as pd 
import numpy as np
import seaborn as sns
from numpy import linalg as la
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

# Import LinearModels.py file 
import LinearModels as lm

# Supress Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Set autoreloads 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Import data
dat = pd.read_csv('firms.csv')

In [17]:
# Defining dimentions 
N = dat.firmid.unique().size
T = dat.year.unique().size
assert dat.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T}')

Data has N=441 and T=12


In [23]:
# Making the data narrays
# a. y
y = dat.ldsa.values.reshape((N*T,1))

# b. x (constant, labour, capital)
ones = np.ones((N*T,1))
l = dat.lemp.values.reshape((N*T,1))
k = dat.lcap.values.reshape((N*T,1))
# i. stack the arrays
x = np.hstack([l, k])   # Disregarding constant term, as data is demeaned

# c. set labels
label_y = 'sales'
label_x = ['labour','capital']

## Estimators

### Pooled OLS (PLOS)

In [24]:
# Estimate coefficients
ols_result = lm.estimate(y, x, '', T = T, N=N)

# Print table
lm.print_table((label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f')

Pooled OLS
Dependent variable: sales

           Beta      Se    t-values
-------  ------  ------  ----------
labour   0.6748  0.0102     66.4688
capital  0.3100  0.0091     33.9269
R² = 0.914
σ² = 0.131


### Fixed Effects (FE) 

In [28]:
# Create transformation matrix
def demeaning_matrix(T):
    Q_T = np.eye(T) - np.tile(1/T,(T,T))
    return Q_T

# Print the matrix
Q_T = demeaning_matrix(T)

# Transform the data
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)

# Create function to check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')

# Check rank of demeaned x
check_rank(x_demean)

# Estimate FE OLS using the demeaned variables.
fe_result = lm.estimate(y_demean, x_demean, transform='fe', N=N, T=T)

# Print results
print('\n')
lm.print_table((label_y, label_x), fe_result, title='FE regression', floatfmt='.4f')

Rank of demeaned x: 2
Eigenvalues of within-transformed x: [ 58. 214.]


FE regression
Dependent variable: sales

           Beta      Se    t-values
-------  ------  ------  ----------
labour   0.6942  0.0147     47.2447
capital  0.1546  0.0130     11.9311
R² = 0.477
σ² = 0.018


### First-difference (FD)

In [31]:
# Create transformation matrix
def fd_matrix(T):
    D_T = np.eye(T, k = 0) - np.eye(T, k=-1)
    D_T = D_T[1:]
    return D_T

# ?
D_T = fd_matrix(T)

# Transform the data.
y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, x)

# Check rank of x_diff
check_rank(x_diff)

# Estimate FE OLS using the demeaned variables.
fd_result = lm.estimate(y_diff, x_diff, transform='fd', N=N, T=T-1)

# Print results
print('\n')
lm.print_table((label_y, label_x), fd_result, title='FD regression', floatfmt='.4f')

Rank of demeaned x: 2
Eigenvalues of within-transformed x: [47. 35.]


FD regression
Dependent variable: sales

           Beta      Se    t-values
-------  ------  ------  ----------
labour   0.5487  0.0183     29.9635
capital  0.0630  0.0191      3.3043
R² = 0.165
σ² = 0.014


### Random Effects (RE)

## Tests

### Strict Exogeneity for FE Estimator

In [35]:
F_T = np.eye(T, k=1)[:-1]

# Lead capital
cap_lead = lm.perm(F_T, x[:, 1].reshape(-1, 1))

# Lead labour
lab_lead = lm.perm(F_T, x[:, 0].reshape(-1, 1))

# Remove the last observed year for every individial
I_T =  np.eye(T, k = 0)[:-1]

x_exo = lm.perm(I_T, x)
y_exo = lm.perm(I_T, y)

# Add cap_lead and lab_lead to x_exo
x_exo = np.hstack((x_exo, cap_lead, lab_lead))

# Within transform the data
Q_T = demeaning_matrix(T-1)

yw_exo = lm.perm(Q_T, y_exo)
xw_exo = lm.perm(Q_T, x_exo)

# Estimate model
exo_test = lm.estimate(yw_exo, xw_exo, transform='fe', N=N, T=T-1)

# Print results
label_exo = label_x + ['Capital lead', 'Labor lead']
lm.print_table((label_y, label_exo), exo_test, title='Exogeneity test', floatfmt='.4f')

# Skal der mere til testen? som fx nogle test-statistics? 

Exogeneity test
Dependent variable: sales

                Beta      Se    t-values
------------  ------  ------  ----------
labour        0.5408  0.0234     23.0904
capital       0.0280  0.0230      1.2153
Capital lead  0.1667  0.0258      6.4706
Labor lead    0.1419  0.0225      6.3134
R² = 0.478
σ² = 0.016
